# Aperiodic Activity

Topic: aperiodic activity is present, and is itself dynamics - making it a tricky confound for many analyses

Sim: demonstate how aperiodic activity, and changes within in, can be mis-interpreted

- FOOOF figure
    - https://fooof-tools.github.io/fooof/auto_motivations/measurements/plot_PeriodicAperiodicFeatures.html
- Filtering aperiodic activity, with changes in aperiodic
    - https://fooof-tools.github.io/fooof/auto_motivations/concepts/plot_IfYouFilterTheyWillCome.html

Core citations / links:

- He et al 2014, FOOOF Paper